In [1]:
!pip install rdkit
!pip install openbabel
!pip install py3Dmolы


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: C:\Users\Таня\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip



  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for openbabel
Failed to build openbabel


  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [15 lines of output]
      running bdist_wheel
      running build
      running build_ext
      Guessing Open Babel location:
      - include_dirs: ['C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\\include', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\\Include', '/usr/local/include/openbabel3']
      - library_dirs: ['C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\\libs', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0', 'C:\\Program Files\\WindowsApps\\PythonSoftwareFoundation.Python.3.11_3.11.2544.0_x64__qbz5n2kfra8p0\\PCbuild\\amd64', '/usr/local/lib']
      building 'openbabel._openbabel' extension
      swigging openbabel\openbabel-python.i to open

In [2]:
import pandas as pd
from rdkit import Chem
from rdkit.Chem import AllChem, Draw, rdFMCS
from rdkit.Chem import rdDepictor
import random
import os
import subprocess
from rdkit import RDLogger


In [3]:

class MoleculeGenerator:
    def __init__(self, num_molecules=300):
        self.num_molecules = num_molecules
        self.core_templates = self._init_core_templates()
        self.fragments = self._init_fragments()
        self.generated_molecules = []  # Для хранения всех сгенерированных молекул

    def _init_core_templates(self):
        """Инициализация коровых структур"""
        cores = [
            "C1CCCCC1", "C1CCCC1", "c1ccccc1", 
            "C1CCNCC1", "C1COCC1", "CCCC",
            "CCC(=O)O", "C1CCCCC1C(=O)O"
        ]
        return [Chem.MolFromSmiles(smi) for smi in cores if Chem.MolFromSmiles(smi)]

    def _init_fragments(self):
        """Инициализация фрагментов"""
        frags = [
                "Cl[*:1]",
        "C#C[*:1]",
        "C(=C[*:1])[*:2]",
        "C([*:1])([*:3])([*:2])[*:4]",
        "C1CC([*:1])C([*:2])C1[*:3]",
        "C1CC([*:2])C([*:1])C([*:3])C1",
        "C1CC([*:2])C([*:1])O1",
        "C1CC([*:3])CC1[NH+]([*:2])[*:1]",
        "C1CC1N[*:1]",
        "C1CC1[*:1]",
        "C1CCCC([*:1])CC1",
        "C1CCN([*:1])C1",
        "C1CN([*:1])CCO1",
        "C1CN([*:2])CC([*:3])C1[*:1]",
        "C1C[NH+]([*:2])CCN1C(C[*:3])[*:1]",
        "C=CCO[*:1]",
        "CC(C(N[*:2])C(=O)[*:3])[*:1]",
        "CC(C(N[*:2])[*:3])[*:1]",
        "CC(C)C([*:2])[*:1]",
        "CC(C)(N[*:2])[*:1]",
        "CC(C)(O[*:2])[*:1]",
        "CC(O[*:2])C(=O)[*:1]",
        "CC(S[*:2])C(=O)[*:1]",
        "CC(S[*:2])[*:1]",
        "CC1C([*:1])CCCC1[*:2]",
        "CCOC(=O)[*:1]",
        "CN(CC(=O)[*:2])[*:1]",
        "CN(C[*:2])[*:1]",
        "COCC[*:1]",
        "COC[*:1]",
        "CS(=O)(=O)[*:1]",
        "CS(=O)[*:1]",
        "Clc1cc([*:2])ccc1[*:1]",
        "FC(C[*:1])([*:2])[*:3]",
        "FC(F)(F)[*:1]",
        "FC(O[*:2])[*:1]",
        "N#C[*:1]",
        "N(=C([*:1])[*:3])[*:2]",
        "N(N[*:2])[*:1]",
        "N([*:1])([*:2])[*:3]",
        "O([*:2])[*:1]",
        "O=C(C(=O)[*:1])[*:2]",
        "O=C(C([*:1])[*:2])[*:3]",
        "O=C(C1C=CC([*:1])C1)[*:2]",
        "O=C(CN([*:1])[*:2])[*:3]",
        "O=C(CS[*:2])N[*:1]",
        "O=C(N([*:1])[*:3])[*:2]",
        "O=C(NC([*:2])[*:1])[*:3]",
        "O=C(N[*:1])C(=O)[*:2]",
        "O=C(N[*:1])N[*:2]",
        "O=C(O[*:2])[*:1]",
        "O=C([*:1])[*:2]",
        "O=C([O-])C1C=C([*:1])C([*:2])C1",
        "O=C([O-])[*:1]",
        "O=C1CCC([*:1])N1[*:2]",
        "O=S(=O)(N([*:2])[*:1])[*:3]",
        "O=S(=O)(N1CCC([*:1])CC1)[*:2]",
        "O=S(=O)(NC[*:1])[*:2]",
        "O=S(=O)([*:1])[*:2]",
        "O=S(=O)(c1ccc([*:2])cc1)[*:1]",
        "O=S([*:1])[*:2]",
        "O=S1(=O)CC([*:1])C([*:2])C1",
        "O=[N+]([O-])c1ccc([*:1])cc1[*:2]",
        "OC1CC([*:1])CC1[*:2]",
        "OC1CN([*:2])CCC1[*:1]",
        "S([*:1])[*:2]",
        "S=C(N[*:1])[*:2]",
        "S=C([*:1])[*:2]",
        "c1c([*:2])noc1[*:1]",
        "c1cc(O[*:1])cc([*:2])c1",
        "c1cc([*:1])co1",
        "c1cc([*:1])sc1[*:2]",
        "c1cc([*:3])c([*:1])cc1[*:2]",
        "c1cc2c(cc1[*:1])OCO2",
        "c1ccc(C([*:1])[*:2])cc1",
        "c1ccc(C[*:1])nc1",
        "c1ccc([*:1])c(N[*:2])c1",
        "c1c([*:1])c2c(cc1[*:2])OCO2",
        "c1c([*:1])cc([*:2])nc1",
        "c1([*:1])cnc([*:2])nc1",
        "c1([*:1])coc([*:2])c1",
        "c1sc([*:2])nc1[*:1]",
        "n1nc([*:1])oc1[*:2]",
        "C1=CC2c3cc([*:1])ccc3NC([*:2])C2C1",
        "C1C2CC3CC1CC([*:1])(C2)C3",
        "c1ccc2c(C([*:2])N[*:1])c[nH]c2c1",
        "Cn1ncc2c1CC([*:1])CC2[*:2]",
        "c1cnc2c([*:1])cccc2c1[*:2]",
        "c1([*:1])ccc2c([*:2])c[nH]c2c1",
        "c1([*:1])ccc2c(c1)CC[NH+]([*:2])C2",
        "c1([*:1])c([*:2])cc2c(c1)OCC([*:3])O2"
            ]
        return [Chem.MolFromSmiles(smi) for smi in frags if Chem.MolFromSmiles(smi)]

    def generate_molecules(self):
        """Генерация молекул с сохранением в список"""
        valid_molecules = []
        attempts = 0
        
        while len(valid_molecules) < self.num_molecules and attempts < self.num_molecules * 3:
            attempts += 1
            try:
                # Выбираем случайный шаблон
                core = random.choice(self.core_templates)
                core = Chem.RWMol(Chem.Mol(core))
                
                # Добавляем 1-2 фрагмента
                for _ in range(random.randint(1, 2)):
                    frag = random.choice(self.fragments)
                    
                    # Находим атомы для соединения
                    core_atoms = [a.GetIdx() for a in core.GetAtoms() if a.GetDegree() < 4]
                    frag_atoms = [a.GetIdx() for a in frag.GetAtoms() if a.GetDegree() < 2]
                    
                    if not core_atoms or not frag_atoms:
                        continue
                    
                    # Соединяем молекулы
                    combo = Chem.CombineMols(core, frag)
                    edcombo = Chem.EditableMol(combo)
                    edcombo.AddBond(
                        random.choice(core_atoms),
                        combo.GetNumAtoms() - frag.GetNumAtoms() + random.choice(frag_atoms),
                        Chem.BondType.SINGLE
                    )
                    
                    # Получаем новую молекулу
                    new_mol = edcombo.GetMol()
                    new_mol = Chem.RemoveHs(new_mol)
                    
                    # Проверяем валидность
                    if new_mol and '.' not in Chem.MolToSmiles(new_mol):
                        core = Chem.RWMol(new_mol)
                
                # Сохраняем молекулу
                if core:
                    final_mol = core.GetMol()
                    Chem.SanitizeMol(final_mol)
                    smiles = Chem.MolToSmiles(final_mol)
                    if len(smiles) > 3:  # Отсеиваем слишком простые структуры
                        valid_molecules.append(smiles)
                        
            except Exception:
                continue
                
        print(f"Сгенерировано {len(valid_molecules)} молекул (из {attempts} попыток)")
        self.generated_molecules = valid_molecules
        return valid_molecules

    def save_molecules_to_csv(self, filename="generated_molecules.csv"):
        """Сохранение всех сгенерированных молекул в CSV"""
        if not self.generated_molecules:
            print("Нет молекул для сохранения")
            return
            
        df = pd.DataFrame({
            'SMILES': self.generated_molecules,
            'Docking_Status': 'Not attempted'  # Добавляем колонку статуса
        })
        
        df.to_csv(filename, index=False)
        print(f"\nСохранено {len(df)} молекул в файл {filename}")


In [4]:

# Пример использования
if __name__ == "__main__":
    # Инициализация генератора
    generator = MoleculeGenerator(num_molecules=300)
    
    # Генерация молекул
    print("1. Генерация молекул...")
    molecules = generator.generate_molecules()
    
    # Сохранение результатов
    print("\n2. Сохранение молекул...")
    generator.save_molecules_to_csv()
    


1. Генерация молекул...


[12:49:29] Explicit valence for atom # 12 O, 3, is greater than permitted
[12:49:29] Explicit valence for atom # 17 O, 3, is greater than permitted
[12:49:29] Explicit valence for atom # 9 O, 3, is greater than permitted
[12:49:29] Explicit valence for atom # 9 N, 4, is greater than permitted
[12:49:29] Explicit valence for atom # 2 C, 5, is greater than permitted
[12:49:29] Explicit valence for atom # 2 O, 3, is greater than permitted
[12:49:29] Can't kekulize mol.  Unkekulized atoms: 6 7 9 10 12 13 14 15 16
[12:49:29] Explicit valence for atom # 6 O, 3, is greater than permitted
[12:49:29] Explicit valence for atom # 6 O, 3, is greater than permitted
[12:49:29] Explicit valence for atom # 7 O, 3, is greater than permitted
[12:49:29] Explicit valence for atom # 6 N, 4, is greater than permitted
[12:49:29] Explicit valence for atom # 5 O, 3, is greater than permitted
[12:49:29] Explicit valence for atom # 5 O, 3, is greater than permitted
[12:49:29] Explicit valence for atom # 19 O, 3,

Сгенерировано 300 молекул (из 461 попыток)

2. Сохранение молекул...

Сохранено 300 молекул в файл generated_molecules.csv


[12:49:30] Explicit valence for atom # 5 O, 3, is greater than permitted
[12:49:30] Explicit valence for atom # 8 O, 3, is greater than permitted
[12:49:30] Explicit valence for atom # 6 C, 5, is greater than permitted
[12:49:30] Explicit valence for atom # 6 O, 3, is greater than permitted
[12:49:30] Explicit valence for atom # 9 O, 3, is greater than permitted
[12:49:30] Explicit valence for atom # 5 O, 3, is greater than permitted
[12:49:30] Can't kekulize mol.  Unkekulized atoms: 6 7 8 12 13
[12:49:30] Explicit valence for atom # 2 C, 5, is greater than permitted
[12:49:30] Explicit valence for atom # 15 F, 2, is greater than permitted
[12:49:30] Explicit valence for atom # 11 N, 5, is greater than permitted
[12:49:30] Explicit valence for atom # 6 C, 5, is greater than permitted
[12:49:30] Explicit valence for atom # 2 O, 3, is greater than permitted
[12:49:30] Explicit valence for atom # 7 C, 5, is greater than permitted
[12:49:30] Explicit valence for atom # 4 N, 4, is greater t